In [15]:
from PIL import Image
import numpy as np, PIL, pandas as pd, json, re, pickle, os
from hashlib import sha1 as hash_fn

import matplotlib.pyplot as plt

from time import time, localtime, asctime, ctime


from ipywidgets import FloatProgress
from IPython.display import Markdown
from stat import S_ISREG, ST_CTIME, ST_MODE

In [16]:
start = 20
end = 40

step_size=20

chunk_file_suffix = [i for i in range(start, end, step_size)]

base_path = "../"

data_dir = "data/intermediate/"
chunking = "chunking/"
hashes = "hashes/"

base_mnist_path = base_path+"mnist/"
by_field_dir = "by_field/"

In [17]:
def read_hash_pickle(filename):
    """
        Read a pickled img hash dictionary, requires filename
        returns image hash based dictionary
    """
    file = open(filename, "rb")
    img_hash_dict = pickle.loads(file.read())
    file.close()
    return img_hash_dict

In [28]:
#get newest filename
def get_newest_pickle():
    path = data_dir+hashes

    hash_pickles = os.listdir(path)

    pickle_stats = {os.stat(path+pickle)[ST_CTIME]:path+pickle for pickle in hash_pickles}
    display(pickle_stats)
    pickle_key_list = [key for key in pickle_stats.keys()]

    pickle_key_list.sort(reverse=True)

    pickle_path = None
    reloaded = None

    start_file = 0

    if len(pickle_key_list)>0:
        pickle_path = pickle_stats[pickle_key_list[0]]
        pickle_name_split = re.split(r"_([0-9]+)", pickle_path)
        start_file = int(pickle_name_split[-2])

    display(Markdown("Loading Img Hash Pickle for {} processed files".format(start_file)))
    
    if pickle_path is not None:
        reloaded = read_hash_pickle(pickle_path)
        
    return reloaded

In [19]:
img_hash_dict = get_newest_pickle()

{1551846379: 'data/intermediate/hashes/by_field_openssl_sha1_Tue_Mar__5_20.26.19_2019_480000.pickle',
 1551846450: 'data/intermediate/hashes/by_field_openssl_sha1_Tue_Mar__5_20.26.19_2019_480000.pickle.zip',
 1551846455: 'data/intermediate/hashes/by_field_openssl_sha1_Tue_Mar__5_20.27.35_2019_490000.pickle',
 1551846525: 'data/intermediate/hashes/by_field_openssl_sha1_Tue_Mar__5_20.28.45_2019_500000.pickle',
 1551846596: 'data/intermediate/hashes/by_field_openssl_sha1_Tue_Mar__5_20.29.56_2019_510000.pickle',
 1551846673: 'data/intermediate/hashes/by_field_openssl_sha1_Tue_Mar__5_20.31.13_2019_520000.pickle',
 1551846741: 'data/intermediate/hashes/by_field_openssl_sha1_Tue_Mar__5_20.32.21_2019_530000.pickle',
 1551846801: 'data/intermediate/hashes/by_field_openssl_sha1_Tue_Mar__5_20.33.21_2019_540000.pickle',
 1551846860: 'data/intermediate/hashes/by_field_openssl_sha1_Tue_Mar__5_20.34.20_2019_550000.pickle',
 1551846921: 'data/intermediate/hashes/by_field_openssl_sha1_Tue_Mar__5_20.35.

Loading Img Hash Pickle for 590000 processed files

In [29]:
df = pd.DataFrame(columns=[i for i in range((128**2))]+["image", "label", "writer_id"])

downsample_factor = 1 #start with no downsampling

for suffix in chunk_file_suffix:
    chunk_file = data_dir+chunking+"all_by_write_files_{}.json".format(suffix)
    
    json_file = open(chunk_file)
    
    chunk_dict = json.loads(json_file.read())

    for key, writer_char in chunk_dict.items():
        #display(key, writer_char)
        for file in writer_char["files"]:
            #display(file)
            this_hasher = hash_fn()
            
            with open(file, "rb") as img:
                this_hasher.update(img.read())
            
            img.close()
            
            digest = this_hasher.hexdigest()
            
            code_label = None
            
            try:
                code_label = img_hash_dict[digest]["code_label"]
            except KeyError as e:
                display(Markdown("hash for {} not found!".format(file)))
                continue

            im = Image.open(file)
            bit2 = im.convert("P", palette=Image.ADAPTIVE, colors=256)
            x = int(im.width/downsample_factor)  # 128
            y = int(im.height/downsample_factor) # 128
            bit2xy = bit2.resize((x, y), resample=PIL.Image.LANCZOS)
            arr = np.array(bit2xy.getdata())
            arr = arr.reshape((x, y))
            #display(arr)
            display(plt.imshow(arr, cmap="gray"))
            row = {"image":arr, "label": code_label, "writer_id":writer_char["writer_id"]}
            
            row_w_pixel_cols = {index_1d: pixel for index_1d, pixel in enumerate(arr)}
            
            for k, v in row.items():
                row_w_pixel_cols[k] = v
            
            df.append(row_w_pixel_cols, ignore_index=True)

df

hash for ../mnist/by_write/hsf_0/f0000_14/c0000_14/c0000_14_00000.png not found!

hash for ../mnist/by_write/hsf_0/f0000_14/c0000_14/c0000_14_00001.png not found!

hash for ../mnist/by_write/hsf_0/f0000_14/c0000_14/c0000_14_00002.png not found!

hash for ../mnist/by_write/hsf_0/f0000_14/c0000_14/c0000_14_00003.png not found!

hash for ../mnist/by_write/hsf_0/f0000_14/c0000_14/c0000_14_00004.png not found!

...

hash for ../mnist/by_write/hsf_0/f0004_09/u0004_09/u0004_09_00013.png not found!

hash for ../mnist/by_write/hsf_0/f0004_09/u0004_09/u0004_09_00014.png not found!

hash for ../mnist/by_write/hsf_0/f0004_09/u0004_09/u0004_09_00015.png not found!

hash for ../mnist/by_write/hsf_0/f0004_09/u0004_09/u0004_09_00016.png not found!

,0,1,2,3,4,5,6,7,8,9,...,16377,16378,16379,16380,16381,16382,16383,image,label,writer_id
